# 添加patch位置信息的json

In [19]:
import json
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm

# ================================================================
# ✅ 统一版本控制变量 —— 只需修改 VERSION 即可自动更新全部路径
# ================================================================
VERSION = "0.3"

BASE_DIR = "/opt/data/private/BlackBox"
SAVE_DIR = f"{BASE_DIR}/save-{VERSION}"
DATA_DIR = f"{BASE_DIR}/data"
COCO_DIR = f"{DATA_DIR}/coco"

# -------------------------- 可调整参数 --------------------------
RATIO = 0.3  # patch缩放比例（默认0.3）
PATCH_PT_PATH = f"{SAVE_DIR}/final_patch/final_patch.pt"
PATCH_PNG_PATH = f"{SAVE_DIR}/final_patch/final_patch.png"
ORIGIN_IMG_DIR = f"{COCO_DIR}/val2017/"
ANNOTATIONS_PATH = f"{COCO_DIR}/annotations/instances_val2017.json"
OUTPUT_DIR = f"{SAVE_DIR}/attack/detection/img/img-patch/"
PATCH_REGION_JSON_PATH = f"{SAVE_DIR}/attack/detection/img/patch_regions.json"
# ----------------------------------------------------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_annotations(annot_path):
    """加载COCO风格标注文件"""
    with open(annot_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data  # 返回完整JSON结构

def load_patch(patch_pt_path, patch_png_path):
    """加载patch（优先pt格式，失败则用png格式），返回BGR格式的patch数组"""
    if os.path.exists(patch_pt_path):
        try:
            patch = torch.load(patch_pt_path, map_location="cpu").numpy()
            if patch.ndim == 3:
                if patch.shape[0] in [1, 3]:
                    patch = np.transpose(patch, (1, 2, 0))
                if patch.max() <= 1.0 and patch.min() >= 0.0:
                    patch = (patch * 255).astype(np.uint8)
            return patch
        except Exception as e:
            print(f"PT格式patch加载失败：{e}，尝试加载PNG格式")

    if os.path.exists(patch_png_path):
        patch = cv2.imread(patch_png_path)
        if patch is not None:
            return patch
    raise FileNotFoundError("未找到有效patch文件（PT或PNG）")

def paste_patch_to_image(origin_img, boxes, patch, ratio):
    """将patch粘贴到原图的每个检测框中心，并记录粘贴坐标"""
    img_h, img_w = origin_img.shape[:2]
    patch_regions = []

    for box in boxes:
        x1, y1, x2, y2 = map(float, box)
        box_center_x = (x1 + x2) / 2
        box_center_y = (y1 + y2) / 2
        box_w = x2 - x1
        box_h = y2 - y1

        patch_target_size = min(box_w, box_h) * ratio
        patch_target_size = int(max(1, patch_target_size))

        scaled_patch = cv2.resize(patch, (patch_target_size, patch_target_size), interpolation=cv2.INTER_LINEAR)
        scaled_h, scaled_w = scaled_patch.shape[:2]

        paste_x = int(box_center_x - scaled_w / 2)
        paste_y = int(box_center_y - scaled_h / 2)

        paste_x_start = max(0, paste_x)
        paste_x_end = min(img_w, paste_x + scaled_w)
        paste_y_start = max(0, paste_y)
        paste_y_end = min(img_h, paste_y + scaled_h)

        patch_x_start = max(0, -paste_x)
        patch_x_end = patch_x_start + (paste_x_end - paste_x_start)
        patch_y_start = max(0, -paste_y)
        patch_y_end = patch_y_start + (paste_y_end - paste_y_start)

        origin_img[paste_y_start:paste_y_end, paste_x_start:paste_x_end] = scaled_patch[
            patch_y_start:patch_y_end, patch_x_start:patch_x_end
        ]

        # ✅ 输出坐标统一为 [x1, y1, x2, y2]（与 COCO boxes 一致）
        patch_region = [
            int(paste_x_start),
            int(paste_y_start),
            int(paste_x_end),
            int(paste_y_end),
        ]
        patch_regions.append(patch_region)

    return origin_img, patch_regions


def main():
    print(f"=== 运行版本: {VERSION} ===")
    print("加载COCO标注文件...")
    coco_data = load_annotations(ANNOTATIONS_PATH)
    image_info_list = coco_data.get("images", [])
    print(f"共加载 {len(image_info_list)} 张图片")

    print("加载patch文件...")
    patch = load_patch(PATCH_PT_PATH, PATCH_PNG_PATH)
    print(f"Patch加载成功，原始尺寸：{patch.shape[:2]}")

    print("开始粘贴patch并记录补丁区域...")
    pbar = tqdm(total=len(image_info_list), desc="处理进度")

    for img_info in image_info_list:
        img_id = img_info["id"]
        img_filename = img_info["file_name"]
        boxes = img_info.get("boxes", [])

        if not boxes:
            img_info["patch_regions"] = []
            pbar.update(1)
            continue

        origin_img_path = os.path.join(ORIGIN_IMG_DIR, img_filename)
        origin_img = cv2.imread(origin_img_path)
        if origin_img is None:
            print(f"⚠️ 无法读取图片 {origin_img_path}，跳过")
            img_info["patch_regions"] = []
            pbar.update(1)
            continue

        # 粘贴 patch 并获取坐标
        img_with_patch, patch_coords = paste_patch_to_image(origin_img, boxes, patch, RATIO)
        img_info["patch_regions"] = patch_coords

        # 保存贴好补丁的图片
        output_path = os.path.join(OUTPUT_DIR, img_filename)
        cv2.imwrite(output_path, img_with_patch)

        pbar.update(1)

    pbar.close()

    # 保存 JSON 文件（包含 patch_regions）
    with open(PATCH_REGION_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(coco_data, f, indent=2, ensure_ascii=False)

    print(f"\n✅ 完成！所有贴好patch的图片保存在：{OUTPUT_DIR}")
    print(f"📄 新JSON文件（含patch位置）已保存：{PATCH_REGION_JSON_PATH}")

if __name__ == "__main__":
    main()


/tmp/ipykernel_49668/2064866366.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  patch = torch.load(patch_pt_path, map_location="cpu").numpy()


=== 运行版本: 0.3 ===
加载COCO标注文件...
共加载 288 张图片
加载patch文件...
Patch加载成功，原始尺寸：(300, 300)
开始粘贴patch并记录补丁区域...


处理进度: 100%|██████████| 288/288 [00:12<00:00, 23.75it/s]


✅ 完成！所有贴好patch的图片保存在：/opt/data/private/BlackBox/save-0.3/attack/detection/img/img-patch/
📄 新JSON文件（含patch位置）已保存：/opt/data/private/BlackBox/save-0.3/attack/detection/img/patch_regions.json
